# Cleaning Data in Python


## 1- Handling Missing Data 



Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default. The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas use the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:

In [1]:
import numpy as np
import pandas as pd

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python None value is also treated as NA in object arrays:


In [3]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

###### Filtering Out Missing Data
There are a few ways to filter out missing data. While you always have the option to
do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful.
On a Series, it returns the Series with only the non-null data and index values:


In [4]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any NAs. dropna by default drops
any row containing a missing value:

In [6]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing how='all' will only drop rows that are all NA:


In [7]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass axis=1:


In [8]:
data[4] = NA
data
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose
you want to keep only rows containing a certain number of observations. You can
indicate this with the thresh argument:

In [10]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.178992,NaN,NaN
1,-0.326453,NaN,NaN
2,0.043422,NaN,-0.543690
3,-0.668867,NaN,1.631512
4,0.823151,-0.029963,1.213487
5,-0.678324,-0.037185,-0.604125
6,0.156808,-0.888644,-0.001631


In [11]:
df.dropna()

,0,1,2
4,0.823151,-0.029963,1.213487
5,-0.678324,-0.037185,-0.604125
6,0.156808,-0.888644,-0.001631


In [12]:
df.dropna(thresh=2)

,0,1,2
2,0.043422,NaN,-0.543690
3,-0.668867,NaN,1.631512
4,0.823151,-0.029963,1.213487
5,-0.678324,-0.037185,-0.604125
6,0.156808,-0.888644,-0.001631


##### Filling In Missing Data
Rather than filtering out missing data (and potentially discarding other data along
with it), you may want to fill in the “holes” in any number of ways. For most pur‐
poses, the fillna method is the workhorse function to use. Calling fillna with a
constant replaces missing values with that value:


In [13]:
df.fillna(0)

,0,1,2
0,-0.178992,0.000000,0.000000
1,-0.326453,0.000000,0.000000
2,0.043422,0.000000,-0.543690
3,-0.668867,0.000000,1.631512
4,0.823151,-0.029963,1.213487
5,-0.678324,-0.037185,-0.604125
6,0.156808,-0.888644,-0.001631


Calling fillna with a dict, you can use a different fill value for each column:


In [14]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.178992,0.500000,0.000000
1,-0.326453,0.500000,0.000000
2,0.043422,0.500000,-0.543690
3,-0.668867,0.500000,1.631512
4,0.823151,-0.029963,1.213487
5,-0.678324,-0.037185,-0.604125
6,0.156808,-0.888644,-0.001631


fillna returns a new object, but you can modify the existing object in-place:


In [15]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.178992,0.000000,0.000000
1,-0.326453,0.000000,0.000000
2,0.043422,0.000000,-0.543690
3,-0.668867,0.000000,1.631512
4,0.823151,-0.029963,1.213487
5,-0.678324,-0.037185,-0.604125
6,0.156808,-0.888644,-0.001631


## 2-Removing Duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an
example:

In [17]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method duplicated returns a boolean Series indicating whether each
row is a duplicate (has been observed in a previous row) or not:


In [18]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, drop_duplicates returns a DataFrame where the duplicated array is
False:

In [19]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can
specify any subset of them to detect duplicates. Suppose we had an additional column
of values and wanted to filter duplicates only based on the 'k1' column:

In [20]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated and drop_duplicates by default keep the first observed value combina‐
tion. Passing keep='last' will return the last one:

In [21]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6
